# Benchmarks

## Initialize

In [1]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
partitions = [0] # [i for i in range(22)]
##### END   ADAPT #####

experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [3]:
model_path = f"{experiment_path}/coxph/models"
model_list =  !ls $model_path

In [4]:
!ls -lt $model_path

total 1024120
-rwxrwx--- 1 loockl posix-nogroup 505069 May 25 11:33 phecode_977-5_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 541028 May 25 11:33 phecode_977-72_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 542957 May 25 11:33 phecode_977-7_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 501827 May 25 11:33 phecode_977-5_Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 533208 May 25 11:33 phecode_969_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 527518 May 25 11:33 phecode_848-2_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 515246 May 25 11:33 phecode_977-4_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 529349 May 25 11:33 phecode_977-72_Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 532108 May 25 11:33 phecode_977_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 589357 May 25 11:33 phecode_832_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 512319 May 25 11:33 phecode_977-41_Age+Sex+Retina_0.p
-rwxrwx--- 1 loockl posix-nogroup 541767 May 25 

In [5]:
import pandas as pd
all_endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

#all_endpoints = sorted(endpoints_all_md.endpoint.to_list())
print(len(all_endpoints))

endpoints_not_overlapping_with_preds = []
#endpoints_not_overlapping_with_preds_md = pd.read_csv(f"{experiment_path}/endpoints_not_overlapping.csv", header=None)
#print(len(endpoints_not_overlapping_with_preds_md))
#endpoints_not_overlapping_with_preds = list(endpoints_not_overlapping_with_preds_md[0].values)

endpoints = []
for c in all_endpoints:
    if c not in endpoints_not_overlapping_with_preds: # this is what i want
        #print('OK    - ',c)
        endpoints.append(c)
    #if c in endpoints_not_overlapping_with_preds: # this is what causes errors!
    #    print('ERROR - ',c)
print(len(endpoints))

498
498


In [6]:
splits = ["train", "valid", 'test'] # "test_left", 'test_right']

In [7]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").query("endpoint==@endpoints").sort_values("endpoint").set_index("endpoint")

In [8]:
from datetime import date
today = str(date.today())

In [9]:
eligable_eids = pd.read_feather(f"{output_path}/eligable_eids_{today}.feather")
eids_dict = eligable_eids.set_index("endpoint")["eid_list"].to_dict()

In [10]:
%env MKL_NUM_THREADS=4
%env NUMEXPR_NUM_THREADS=4
%env OMP_NUM_THREADS=4

env: MKL_NUM_THREADS=4
env: NUMEXPR_NUM_THREADS=4
env: OMP_NUM_THREADS=4


In [11]:
ray.shutdown()

NameError: name 'ray' is not defined

In [12]:
import ray

ray.init(num_cpus=24)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

{'node_ip_address': '10.32.105.6',
 'raylet_ip_address': '10.32.105.6',
 'redis_address': '10.32.105.6:31561',
 'object_store_address': '/tmp/ray/session_2022-05-25_11-34-21_533767_3773602/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-25_11-34-21_533767_3773602/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-05-25_11-34-21_533767_3773602',
 'metrics_export_port': 57737,
 'node_id': '1a9193edc559e5b7a1554e78cd3b752d409459c08d8b89f9e8db54bc'}

# Predict COX

In [13]:
in_path = pathlib.Path(f"{experiment_path}/coxph/input")
model_path = f"{experiment_path}/coxph/models"

out_path = f"{experiment_path}/coxph/predictions"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [14]:
models = [f.name for f in in_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
models

['ImageTraining_[]_ConvNeXt_MLPHead']

In [15]:
AgeSex = ["age_at_recruitment_f21022_0_0", "sex_f31_0_0"]

In [16]:
from lifelines import CoxPHFitter
from lifelines.exceptions import ConvergenceError
import zstandard
import pickle
import os

def get_features(endpoint):
    features = {
        models[0]: { # TODO CHANGE!
            "Age+Sex": AgeSex,
            "Retina": [endpoint],
            "Age+Sex+Retina": AgeSex + [endpoint],
            #"Age+Sex+MedicalHistory+I(Age*MH)": AgeSex + [endpoint]
            }
    }
    return features

def get_test_data(in_path, partition, models, mapping):
    data = {model: pd.read_feather(f"{in_path}/{model}/{partition}/test.feather").set_index("eid").replace(mapping)for model in models}
    return data
    #left_data = {model: pd.read_feather(f"{in_path}/{model}/{partition}/test_left.feather").set_index("eid").replace(mapping)for model in models}
    #right_data = {model: pd.read_feather(f"{in_path}/{model}/{partition}/test_right.feather").set_index("eid").replace(mapping)for model in models}
    #return (left_data, right_data)
            
def load_pickle(fp):
    with open(fp, "rb") as fh:
        dctx = zstandard.ZstdDecompressor()
        with dctx.stream_reader(fh) as decompressor:
            data = pickle.loads(decompressor.read())
    return data

def predict_cox(cph, data_endpoint, endpoint, feature_set, partition, pred_path):
    times = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    time_cols = {t: f"Ft_{t}" for t in times}
    
    if feature_set=="Age+Sex+MedicalHistory+I(Age*MH)":
        data_endpoint.columns = [c.replace("-", "") for c in data_endpoint.columns]
    
    surv_test = 1-cph.predict_survival_function(data_endpoint, times=times)
    temp_pred = data_endpoint.reset_index()[["eid"]].assign(endpoint=endpoint, features=feature_set, partition=partition)
    for t, col in time_cols.items(): 
        temp_pred[col] = surv_test.T[t].to_list()
    
    temp_pred.to_feather(f"{out_path}/{endpoint}_{feature_set}_{partition}.feather")

# for both eyes
def predict_cox_both_eyes(cph, data_endpoint_left, data_endpoint_right, endpoint, feature_set, partition, pred_path):
    times = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    time_cols = {t: f"Ft_{t}" for t in times}
    
    if feature_set=="Age+Sex+MedicalHistory+I(Age*MH)":
        data_endpoint_left.columns = [c.replace("-", "") for c in data_endpoint_left.columns]
        data_endpoint_right.columns = [c.replace("-", "") for c in data_endpoint_right.columns]

    # left eye
    surv_test_left = 1-cph.predict_survival_function(data_endpoint_left, times=times)
    temp_pred_left = data_endpoint_left.reset_index()[["eid"]].assign(endpoint=endpoint, features=feature_set, partition=partition)
    for t, col in time_cols.items(): 
        temp_pred_left[col] = surv_test_left.T[t].to_list()
    
    temp_pred_left.to_feather(f"{out_path}/{endpoint}_{feature_set}_{partition}_left.feather")
    
    # right eye
    surv_test_right = 1-cph.predict_survival_function(data_endpoint_right, times=times)
    temp_pred_right = data_endpoint_right.reset_index()[["eid"]].assign(endpoint=endpoint, features=feature_set, partition=partition)
    for t, col in time_cols.items(): 
        temp_pred_right[col] = surv_test_right.T[t].to_list()
    
    temp_pred_right.to_feather(f"{out_path}/{endpoint}_{feature_set}_{partition}_right.feather")
    
    # mean both eyes
    eids = list(surv_test_left.columns.values)
    surv_test_mean = pd.concat((surv_test_left, surv_test_right), axis=1)
    surv_test_mean.columns = [f'{col}_left' for col in list(surv_test_left.columns.values)] + [f'{col}_right' for col in list(surv_test_right.columns.values)]
    for eid in eids:
        eid_columns = [col for col in list(surv_test_mean.columns.values) if str(eid) in col]
        surv_test_mean[f'{eid}_mean'] = surv_test_mean[eid_columns].mean(axis=1)
    surv_test_mean = surv_test_mean[[col for col in list(surv_test_mean.columns.values) if 'mean' in col]].rename(columns={col: col.replace('_mean', '') for col in list(surv_test_mean.columns.values)})
    
    temp_pred_mean = data_endpoint_left.reset_index()[["eid"]].assign(endpoint=endpoint, features=feature_set, partition=partition)
    for t, col in time_cols.items(): 
        temp_pred_mean[col] = surv_test_mean.T[t].to_list()
     
    temp_pred_mean.to_feather(f"{out_path}/{endpoint}_{feature_set}_{partition}_mean.feather")

@ray.remote
def predict_endpoint(data_partition, eids_dict, endpoint, partition, models, features, model_path, out_path):
    #data_partition_left, data_partition_right = data_partition
    eids_incl = eids_dict[endpoint].tolist()
    results = []
    for model in models:
        data_model = data_partition[model]
        #data_model_left = data_partition_left[model]
        #data_model_right = data_partition_right[model]
        for feature_set, covariates in features[model].items():
            identifier = f"{endpoint}_{feature_set}_{partition}"
            pred_path = f"{out_path}/{identifier}.feather"
            if not os.path.isfile(pred_path):
                try:
                    cph = load_pickle(f"{model_path}/{identifier}.p")
                    data_endpoint = data_model[data_model.index.isin(eids_incl)]
                    #data_endpoint_left = data_model_left[data_model_left.index.isin(eids_incl)]
                    #data_endpoint_right = data_model_right[data_model_right.index.isin(eids_incl)]
                    predict_cox(cph, data_endpoint, endpoint, feature_set, partition, pred_path)
                    #predict_cox_both_eyes(cph, data_endpoint_left, data_endpoint_right, endpoint, feature_set, partition, pred_path)
                except FileNotFoundErrorundError:
                    print(f"{identifier} not available")
    return True

In [17]:
mapping = {"sex_f31_0_0": {"Female":0, "Male":1}}

ray_eids = ray.put(eids_dict)
for partition in tqdm(partitions): # TODO: CHANGE!
    try:
        del ray_partition
    except:
        print("Ray object not yet initialised")
    ray_partition = ray.put(get_test_data(in_path, partition, models, mapping))
    progress = []
    for endpoint in endpoints:
        features = get_features(endpoint)
        progress.append(predict_endpoint.remote(ray_partition, ray_eids, endpoint, partition, models, features, model_path, out_path))
    [ray.get(s) for s in tqdm(progress)]

  0%|          | 0/1 [00:00<?, ?it/s]

Ray object not yet initialised


  0%|          | 0/498 [00:00<?, ?it/s]